In [1]:
!python 1.py

final cost: 51314.71550400719


In [2]:
import numpy as np
import csv

cities = []
with open('TSP.csv', mode='r', newline='') as tsp:
    reader = csv.reader(tsp)
    for row in reader:
        cities.append(row)

In [27]:
import math

def distance(x, y):
    dist = np.linalg.norm(np.array(x)- np.array(y))
    #dist = math.sqrt(((y[0] - x[0]) ** 2 + (y[1] - x[1]) **2))
    return dist

def getFitness(chrome):
    cost = 0
    for idx in range(len(cities)-1):
        city1 = [float(cities[chrome[idx]][0]), float(cities[chrome[idx]][1])]
        city2 = [float(cities[chrome[idx+1]][0]), float(cities[chrome[idx+1]][1])]

        dist = distance(city1, city2)
        cost += dist
        
    return (1000000/cost) ** 2

In [38]:
import random

def selectChrom(fits, total):
    rand = random.random()
    sum = 0
    ans = 0
    #arr = np.array(fits)
    arr = fits.copy()
    
    for i in range(len(arr)):
        sum += arr[i] / total;
        if sum >= rand:
            ans = i
            break
    ret = arr.index(arr[ans])
    return ret

def getRouletteChrom(fits, total):
    ch1 = selectChrom(fits, total)
    ch2 = ch1
    while ch1 == ch2:
        ch2 = selectChrom(fits, total)
    return ch1, ch2

In [77]:
def newChrom(idx1, idx2, chrom_list, fit_list):
    #point = int(random.random() * 1000)
    chrom1 = chrom_list[idx1]
    chrom2 = chrom_list[idx2]
    fit1 = fit_list[idx1]
    fit2 = fit_list[idx2]
    point = int(len(chrom1) * (fit1 / (fit1 + fit2)))
    left = []
    blank_idx = []
    child = []
    for i in range(len(chrom1)):
        left.append(i)
    for i in range(point):
        child.append(chrom1[i])
        left[chrom1[i]] = -1
    for i in range(point, len(chrom2)):
        if left.count(chrom2[i]) == 0:
            blank_idx.append(i)
            child.append(-1)
            continue
        child.append(chrom2[i])
        left[chrom2[i]] = -1
    j = 0
    for i in blank_idx:
        for k in range(j, len(left)):
            if left[k] != -1:
                j = k
                break
        child[i] = left[j]
        left[j] = -1
    #child = np.concatenate((ch1[:point], ch2[point:]), axis=0)
    return child

In [78]:
def mutate(chrom):
    chrom = chrom.copy()
    if random.random() > MUTATION_RATE:
        return chrom
    
    sel1 = int(random.random()*1000000) % len(cities)
    sel2 = int(random.random()*1000000) % len(cities)
    idx1 = chrom.index(sel1)
    idx2 = chrom.index(sel2)
    
    chrom[idx1] = sel2
    chrom[idx2] = sel1
    return chrom

In [81]:
# Init chromosome

chrom_list = []
CHROM_COUNT = 64
MUTATION_RATE = 0.05

for i in range(CHROM_COUNT):
    chrom_list.append(np.random.permutation(len(cities)))
    
fit_list = []
fit_total = 0
for i in range(CHROM_COUNT):
    fit_list.append(getFitness(chrom_list[i]))
    fit_total += fit_list[i]

In [82]:
# GA
best_chrom = []
best_fitness = 0

try:
    with open('best.csv', mode='r', newline='') as file:
        reader = csv.reader(file)
        for i, val in enumerate(reader):
            if i == 0:
                best_fitness = float(val[0])
            else:
                best_chrom.append(int(val[0]))
except:
    print('failed to load')
    best_chrom = []
    best_fitness = 0

print('best fitness : ', best_fitness)


for j in range(101):
    new_chrom_list = []
    new_fit_total = 0
    new_fit_list = []
    
    
    if j%10 == 0:
        print('#', j)
        
    for i in range(CHROM_COUNT):
        idx1, idx2 = getRouletteChrom(fit_list, fit_total)
        child = newChrom(idx1, idx2, chrom_list, fit_list)
            
        child = mutate(child)
        new_chrom_list.append(child)
        fitness = getFitness(child)
        new_fit_list.append(fitness)
        new_fit_total += fitness
        
        if (best_fitness < fitness):
            best_chrom = child.copy()
            best_fitness = fitness
            print('#', j)
            print('best fitness : ', fitness)
        
    arr = np.array(new_fit_list)
    idx = np.argmax(arr)
    print('#', j, ' local best : ', arr[idx])
    fit_list = new_fit_list.copy()
    fit_total = new_fit_total.copy()
    chrom_list = new_chrom_list.copy()
    
# svae best
with open('best.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([float(best_fitness)])
    for val in best_chrom:
        writer.writerow([int(val)])


best fitness :  411.7362907231847
# 0
# 0  local best :  387.6303549995374
# 1  local best :  389.03969322266045
# 2  local best :  382.95978646095585
# 3  local best :  389.06761049098975
# 4  local best :  386.060608188689
# 5  local best :  400.4045230726288
# 6  local best :  398.1460348725114
# 7  local best :  392.14617607968074
# 8  local best :  396.4018599797688
# 9  local best :  399.9166997855132
# 10
# 10  local best :  394.3687034313291
# 11  local best :  396.2167717713432
# 12  local best :  392.9810977868296
# 13  local best :  391.21428845444365
# 14  local best :  392.9638075339221
# 15  local best :  389.2519415988547
# 16  local best :  387.1902013983642
# 17  local best :  395.09444148690756
# 18  local best :  392.82281001443573
# 19  local best :  387.09783512253404
# 20
# 20  local best :  389.80211364848634
# 21  local best :  391.0090409434767
# 22  local best :  392.38831275976054
# 23  local best :  398.454882157286
# 24  local best :  398.4791453579586
# 25